# Pattern Analysis Tutorial

This notebook demonstrates the advanced pattern analysis features in Cancrizans.

## Overview

The pattern analysis module provides:
- Motif detection
- Melodic sequence identification
- Imitation point detection
- Thematic development analysis
- Contour similarity finding
- Fugue structure analysis
- Voice independence metrics
- Pattern complexity calculation

In [ ]:
# Import required modules
from music21 import stream, note, chord
from cancrizans import (
    detect_motifs,
    identify_melodic_sequences,
    detect_imitation_points,
    analyze_thematic_development,
    find_contour_similarities,
    analyze_fugue_structure,
    calculate_voice_independence,
    calculate_pattern_complexity
)
import matplotlib.pyplot as plt
import numpy as np

# Set up plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 4)

## 1. Motif Detection

Detect recurring melodic and rhythmic patterns in a musical stream.

In [ ]:
# Create a melody with repeating motifs
melody = stream.Stream()

# Motif 1: C-D-E pattern (repeated 3 times)
for i in range(3):
    melody.append(note.Note('C4', quarterLength=1))
    melody.append(note.Note('D4', quarterLength=1))
    melody.append(note.Note('E4', quarterLength=1))
    if i < 2:
        melody.append(note.Note('F4', quarterLength=1))

# Detect motifs
motifs = detect_motifs(melody, min_length=3, min_occurrences=2)

print(f"Found {len(motifs)} motifs:\n")
for i, motif in enumerate(motifs[:5], 1):
    print(f"Motif {i}:")
    print(f"  Intervals: {motif.intervals}")
    print(f"  Rhythms: {motif.rhythms}")
    print(f"  Occurrences: {len(motif.occurrences)} times")
    print(f"  Positions: {motif.occurrences}")
    print()

## 2. Melodic Sequences

Identify sequential patterns where a motif is repeated at different pitch levels.

In [ ]:
# Create a melodic sequence (ascending)
sequence_melody = stream.Stream()

# Pattern repeated at ascending steps
for base_pitch in [60, 62, 64, 65]:  # C, D, E, F
    sequence_melody.append(note.Note(base_pitch, quarterLength=1))
    sequence_melody.append(note.Note(base_pitch + 2, quarterLength=1))

# Identify sequences
sequences = identify_melodic_sequences(sequence_melody, min_repetitions=2)

print(f"Found {len(sequences)} melodic sequences:\n")
for i, seq in enumerate(sequences, 1):
    print(f"Sequence {i}:")
    print(f"  Type: {seq['type']}")
    print(f"  Pattern: {seq['pattern']}")
    print(f"  Transpositions: {seq['transpositions']}")
    print(f"  Repetitions: {seq['repetitions']}")
    print()

## 3. Imitation Detection

Find points where voices imitate each other in polyphonic music.

In [ ]:
# Create a two-voice canon with imitation
canon = stream.Score()

# Voice 1 (leader)
voice1 = stream.Part()
for pitch in [60, 62, 64, 65, 64, 62]:  # C D E F E D
    voice1.append(note.Note(pitch, quarterLength=1))

# Voice 2 (follower, delayed by 2 beats)
voice2 = stream.Part()
voice2.append(note.Rest(quarterLength=2))
for pitch in [60, 62, 64, 65]:  # C D E F
    voice2.append(note.Note(pitch, quarterLength=1))

canon.append(voice1)
canon.append(voice2)

# Detect imitation
imitations = detect_imitation_points(canon, time_window=4.0)

print(f"Found {len(imitations)} imitation points:\n")
for i, im in enumerate(imitations[:3], 1):
    print(f"Imitation {i}:")
    print(f"  Leader voice: {im['leader_voice']}")
    print(f"  Follower voice: {im['follower_voice']}")
    print(f"  Delay: {im['delay']} quarter notes")
    print(f"  Similarity: {im['similarity']:.2f}")
    print(f"  Type: {im['type']}")
    print()

## 4. Thematic Development

Track how themes evolve throughout a piece.

In [ ]:
# Create a piece with thematic development
piece = stream.Stream()

# Theme appears 3 times with variations
for i in range(3):
    for pitch in [60, 64, 67, 72]:  # C E G C (major triad)
        piece.append(note.Note(pitch, quarterLength=1))
    # Gap between repetitions
    for _ in range(4):
        piece.append(note.Note(55, quarterLength=0.5))

# Analyze thematic development
development = analyze_thematic_development(piece, theme_length=4)

print(f"Thematic Analysis:\n")
print(f"Number of themes: {development['theme_count']}")
print(f"Number of transformations: {len(development['transformations'])}")
print(f"Development sections: {len(development['development_sections'])}")
print(f"Recapitulations: {len(development['recapitulations'])}")
print()

for i, theme in enumerate(development['themes'][:3], 1):
    print(f"Theme {i}:")
    print(f"  Intervals: {theme['intervals']}")
    print(f"  Occurrences: {len(theme['occurrences'])} times")
    print(f"  First appearance: offset {theme['first_appearance']}")
    print()

## 5. Contour Similarity

Find melodic contours with similar shapes regardless of exact intervals.

In [ ]:
# Create melodies with similar contours
contour_piece = stream.Stream()

# Contour: up, up, down (twice with different intervals)
# First occurrence: C D E D (intervals +2, +2, -2)
contour_piece.append(note.Note(60, quarterLength=1))  # C
contour_piece.append(note.Note(62, quarterLength=1))  # D (up)
contour_piece.append(note.Note(64, quarterLength=1))  # E (up)
contour_piece.append(note.Note(62, quarterLength=1))  # D (down)

# Second occurrence: F G A G (intervals +2, +2, -2 - same contour!)
contour_piece.append(note.Note(65, quarterLength=1))  # F
contour_piece.append(note.Note(67, quarterLength=1))  # G (up)
contour_piece.append(note.Note(69, quarterLength=1))  # A (up)
contour_piece.append(note.Note(67, quarterLength=1))  # G (down)

# Find contour similarities
contours = find_contour_similarities(contour_piece, min_length=4)

print(f"Found {len(contours)} matching contours:\n")
for i, c in enumerate(contours[:5], 1):
    print(f"Contour {i}:")
    print(f"  Shape: {c['contour']}")
    print(f"  Length: {c['length']} notes")
    print(f"  Occurrences: {c['occurrences']} times")
    print(f"  Positions: {c['offsets']}")
    print()

## 6. Fugue Structure Analysis

Analyze fugue structure including subject, answers, and episodes.

In [ ]:
# Create a simple fugue-like structure
fugue = stream.Score()

# Subject in voice 1
voice1 = stream.Part()
subject_pitches = [60, 62, 64, 65, 67, 65]
for pitch in subject_pitches:
    voice1.append(note.Note(pitch, quarterLength=1))
# Continue voice 1
for _ in range(6):
    voice1.append(note.Note(55, quarterLength=1))

# Answer in voice 2 (delayed, at the fifth)
voice2 = stream.Part()
voice2.append(note.Rest(quarterLength=3))
for pitch in [67, 69, 71, 72, 74, 72]:  # Transposed up a fifth
    voice2.append(note.Note(pitch, quarterLength=1))
# Continue voice 2
for _ in range(3):
    voice2.append(note.Note(62, quarterLength=1))

fugue.append(voice1)
fugue.append(voice2)

# Analyze fugue structure
fugue_analysis = analyze_fugue_structure(fugue, subject_length=6)

print("Fugue Structure Analysis:\n")
if fugue_analysis['subject']:
    print(f"Subject:")
    print(f"  Intervals: {fugue_analysis['subject']['intervals']}")
    print(f"  Length: {fugue_analysis['subject']['length']} notes")
    print()

print(f"Answers: {len(fugue_analysis['answers'])}")
for ans in fugue_analysis['answers']:
    print(f"  Voice {ans['voice']}, offset {ans['offset']}, type: {ans['type']}")
print()

print(f"Counter-subjects: {len(fugue_analysis['counter_subjects'])}")
print(f"Episodes: {len(fugue_analysis['episodes'])}")
print(f"Stretto sections: {len(fugue_analysis['stretto_sections'])}")
print(f"Exposition ends at: {fugue_analysis['exposition_end']}")

## 7. Voice Independence Metrics

Calculate independence metrics for polyphonic voices.

In [ ]:
# Create a three-voice polyphonic piece
polyphony = stream.Score()

# Voice 1 - mostly quarter notes
v1 = stream.Part()
for pitch in [60, 62, 64, 65, 64, 62, 60, 59]:
    v1.append(note.Note(pitch, quarterLength=1))

# Voice 2 - eighth notes, rhythmically independent
v2 = stream.Part()
v2.append(note.Rest(quarterLength=0.5))
for pitch in [55, 57, 59, 60, 62, 64, 65, 67, 69, 67, 65, 64, 62, 60, 59]:
    v2.append(note.Note(pitch, quarterLength=0.5))

# Voice 3 - half notes
v3 = stream.Part()
for pitch in [48, 50, 52, 53]:
    v3.append(note.Note(pitch, quarterLength=2))

polyphony.append(v1)
polyphony.append(v2)
polyphony.append(v3)

# Calculate voice independence
independence = calculate_voice_independence(polyphony)

print("Voice Independence Metrics:\n")
print(f"Number of voices: {independence['num_voices']}")
print(f"Rhythmic independence: {independence['rhythmic_independence']:.2f}")
print(f"Melodic independence: {independence['melodic_independence']:.2f}")
print(f"Contour independence: {independence['contour_independence']:.2f}")
print(f"Harmonic density: {independence['harmonic_density']:.2f} voices")
print(f"Voice crossings: {independence['voice_crossing_count']}")
print()
print("Per-voice activity:")
for i, activity in enumerate(independence['per_voice_activity'], 1):
    print(f"  Voice {i}: {activity:.2f}")

# Visualize independence metrics
metrics = ['Rhythmic\nIndep.', 'Melodic\nIndep.', 'Contour\nIndep.']
values = [
    independence['rhythmic_independence'],
    independence['melodic_independence'],
    independence['contour_independence']
]

plt.figure(figsize=(10, 5))
plt.bar(metrics, values, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.ylabel('Independence Score (0-1)')
plt.title('Voice Independence Metrics')
plt.ylim(0, 1.1)
plt.grid(axis='y', alpha=0.3)
for i, v in enumerate(values):
    plt.text(i, v + 0.05, f'{v:.2f}', ha='center', fontweight='bold')
plt.tight_layout()
plt.savefig('../docs/images/voice_independence_metrics.png', dpi=100, bbox_inches='tight')
plt.show()
print("\n✓ Saved visualization to docs/images/voice_independence_metrics.png")

## 8. Pattern Complexity

Calculate complexity metrics for musical patterns.

In [ ]:
# Create patterns with different complexity levels
simple_pattern = stream.Stream()
for pitch in [60, 62, 60, 62, 60, 62]:  # Simple, repetitive
    simple_pattern.append(note.Note(pitch, quarterLength=1))

complex_pattern = stream.Stream()
rhythms = [1, 0.5, 0.5, 1.5, 0.5, 2, 0.25, 0.25, 0.5]
pitches = [60, 65, 59, 72, 55, 67, 62, 69, 58]  # Varied intervals and range
for pitch, rhythm in zip(pitches, rhythms):
    complex_pattern.append(note.Note(pitch, quarterLength=rhythm))

# Calculate complexity
simple_complexity = calculate_pattern_complexity(simple_pattern)
complex_complexity = calculate_pattern_complexity(complex_pattern)

print("Pattern Complexity Comparison:\n")
print("Simple Pattern:")
for key, value in simple_complexity.items():
    print(f"  {key}: {value:.3f}" if isinstance(value, float) else f"  {key}: {value}")

print("\nComplex Pattern:")
for key, value in complex_complexity.items():
    print(f"  {key}: {value:.3f}" if isinstance(value, float) else f"  {key}: {value}")

# Visualize complexity comparison
categories = ['Interval\nComplexity', 'Rhythmic\nComplexity', 'Range\nComplexity', 'Overall\nComplexity']
simple_vals = [
    simple_complexity['interval_complexity'],
    simple_complexity['rhythmic_complexity'],
    simple_complexity['range_complexity'],
    simple_complexity['overall_complexity']
]
complex_vals = [
    complex_complexity['interval_complexity'],
    complex_complexity['rhythmic_complexity'],
    complex_complexity['range_complexity'],
    complex_complexity['overall_complexity']
]

x = np.arange(len(categories))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 5))
bars1 = ax.bar(x - width/2, simple_vals, width, label='Simple Pattern', color='#1f77b4')
bars2 = ax.bar(x + width/2, complex_vals, width, label='Complex Pattern', color='#ff7f0e')

ax.set_ylabel('Complexity Score (0-1)')
ax.set_title('Pattern Complexity Comparison')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()
ax.set_ylim(0, 1.1)
ax.grid(axis='y', alpha=0.3)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('../docs/images/pattern_complexity_comparison.png', dpi=100, bbox_inches='tight')
plt.show()
print("\n✓ Saved visualization to docs/images/pattern_complexity_comparison.png")

## Summary

This tutorial demonstrated all 8 pattern analysis functions:

1. **Motif Detection** - Find recurring patterns with sliding window analysis
2. **Melodic Sequences** - Identify sequential patterns at different pitch levels
3. **Imitation Detection** - Find imitative entries between voices
4. **Thematic Development** - Track theme evolution and transformations
5. **Contour Similarity** - Find similar melodic shapes
6. **Fugue Analysis** - Analyze fugue structure (subject, answer, episodes)
7. **Voice Independence** - Calculate polyphonic independence metrics
8. **Pattern Complexity** - Measure musical pattern complexity

These tools are useful for:
- Music analysis and research
- Compositional studies
- Fugue and canon analysis
- Pedagogical applications
- Automated music understanding